In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'book-recommendation-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1004280%2F7595263%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240602%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240602T145817Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D25ec1e1c9ed66652bdbabe7740fb8b78efd5d6fa5fff838546e601590cb08c7f1eb7e7221ca6b4247546506e410b5fa39b29faf4f657df5241652dc2d3443a2b6f8833577277b5451f5f0d99fe6ce4d74b65f2e369049085911da38c7169c861e1dd4e164386a5b068744e0a8f3da1420d9fff47e6289cb2a2cfa91f2ee21ee72fbbedd839919c33ae9ce49d151fb3b422a3330ab8af12ad094ba5754da800375113dfc00ca00793611c929d37ecfe6b65a20d6d2270f991efc323caef7315697a30b9aeaa951c6e8bd31453e82a867f74f9b356ba07d9341b8e37ed3417b02f80d15e110ac46582bcbe833fd66071593d5bf553f5b9b68e0a35c6073201a69a'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
book_df = pd.read_csv('../input/book-recommendation-dataset/Books.csv')
ratings_df = pd.read_csv('../input/book-recommendation-dataset/Ratings.csv').sample(40000)
user_df = pd.read_csv('../input/book-recommendation-dataset/Users.csv')
user_rating_df = ratings_df.merge(user_df, left_on = 'User-ID', right_on = 'User-ID')

/tmp/ipykernel_33/2078159047.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book_df = pd.read_csv('../input/book-recommendation-dataset/Books.csv')


In [ ]:
book_user_rating = book_df.merge(user_rating_df, left_on = 'ISBN',right_on = 'ISBN')
book_user_rating = book_user_rating[['ISBN', 'Book-Title', 'Book-Author', 'User-ID', 'Book-Rating']]
book_user_rating.reset_index(drop=True, inplace = True)

In [ ]:
d ={}
for i,j in enumerate(book_user_rating.ISBN.unique()):
    d[j] =i
book_user_rating['unique_id_book'] = book_user_rating['ISBN'].map(d)

In [ ]:
users_books_pivot_matrix_df = book_user_rating.pivot(index='User-ID',
                                                          columns='unique_id_book',
                                                          values='Book-Rating').fillna(0)

In [ ]:
users_books_pivot_matrix_df.head()

unique_id_book,0,1,2,3,4,5,6,7,8,9,...,26162,26163,26164,26165,26166,26167,26168,26169,26170,26171
User-ID,,,,,,,,,,,,,,,,,,,,,
8,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
users_books_pivot_matrix_df = users_books_pivot_matrix_df.values
users_books_pivot_matrix_df

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
from scipy.sparse.linalg import svds

NUMBER_OF_FACTORS_MF = 15

#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(users_books_pivot_matrix_df, k = NUMBER_OF_FACTORS_MF)

In [ ]:
sigma = np.diag(sigma)
sigma.shape

(15, 15)

In [ ]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)
all_user_predicted_ratings

array([[-3.53426282e-34,  2.05025336e-18,  1.69885615e-37, ...,
         0.00000000e+00,  3.93315082e-21,  0.00000000e+00],
       [-7.27855150e-34,  5.22277411e-18,  2.34980874e-37, ...,
         0.00000000e+00,  4.74508914e-21,  0.00000000e+00],
       [-1.38644424e-36,  6.03611251e-21,  1.18386069e-40, ...,
         0.00000000e+00,  4.33856388e-24,  0.00000000e+00],
       ...,
       [-5.83513074e-35, -7.48260954e-19,  1.08362226e-38, ...,
         0.00000000e+00, -6.08409210e-22,  0.00000000e+00],
       [ 8.53696925e-34, -3.08331753e-18, -9.40115427e-38, ...,
         0.00000000e+00, -3.97355693e-21,  0.00000000e+00],
       [-2.70566597e-34, -3.53586912e-18,  6.80063204e-38, ...,
         0.00000000e+00, -2.81258855e-21,  0.00000000e+00]])

In [ ]:
def top_cosine_similarity(data, book_id, top_n=10):
    index = book_id
    book_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

def similar_books(book_user_rating, book_id, top_indexes):
    print('Recommendations for {0}: \n'.format(
    book_user_rating[book_user_rating.unique_id_book == book_id]['Book-Title'].values[0]))
    for id in top_indexes + 1:
        print(book_user_rating[book_user_rating.unique_id_book == id]['Book-Title'].values[0])

In [ ]:
k = 50
movie_id =25954
top_n = 3
sliced = Vt.T[:, :k] # representative data

similar_books(book_user_rating, 25954, top_cosine_similarity(sliced, movie_id, top_n))

Recommendations for Meditation for Beginners (Headway Guides for Beginners): 

Doctored Evidence
Meditation for Beginners (Headway Guides for Beginners)
The Tipping Point: How Little Things Can Make a Big Difference


/tmp/ipykernel_33/437162856.py:5: RuntimeWarning: invalid value encountered in divide
  similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)
